In [1]:
# import the libraries
import pandas as pd
import numpy as np
from scipy.io.arff import loadarff
from matplotlib import pyplot as plt

### Preprocess EEG Eye State Dataset¶

In [2]:
# 14980 samples, 14 attrs, last column binary label

# load the data. The function loadarff read most arff files and it can also read
# files with missing data, representing the data points as NaNs. This 
# information is important for data preprocessing. The data used here 
# has no missing values
EEG_Eye_State, meta = loadarff('EEG-Eye-State.arff')

In [3]:
# meta contains information about the arff file, as shown below is the attributes
meta

Dataset: EEG_DATA
	AF3's type is numeric
	F7's type is numeric
	F3's type is numeric
	FC5's type is numeric
	T7's type is numeric
	P7's type is numeric
	O1's type is numeric
	O2's type is numeric
	P8's type is numeric
	T8's type is numeric
	FC6's type is numeric
	F4's type is numeric
	F8's type is numeric
	AF4's type is numeric
	eyeDetection's type is nominal, range is ('0', '1')

In [4]:
# EEG_Eye_State records the data of the arff file, accessible by attribute names
# When add the EEG_Eye_State data to matrix, each element in the matrix has the type numpy.bytes_, therefore need to convert to
# float or int type so data matrix could be manipulated without errors
# Turn EEG_Eye_State into matrix of data
Eye_State_data = np.array(EEG_Eye_State[meta.names()[0]].astype(float, copy = True)).reshape(14980,1)

# Load attributes as type float
for i in range(1,14):
    Eye_State_data = np.c_[Eye_State_data, np.array(EEG_Eye_State[meta.names()[i]]).astype(float, copy = True)]

# Load label as type int
Eye_State_data = np.c_[Eye_State_data, np.array(EEG_Eye_State[meta.names()[14]]).astype(int, copy = True)]

# Convert to pandas DataFrame for easier manipulation 
df = pd.DataFrame(data = Eye_State_data, columns = meta.names()[:])

# First 10 samples
df.head(10)

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0.0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0.0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0.0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0.0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0.0
5,4321.03,4004.62,4284.10,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74,0.0
6,4319.49,4001.03,4280.51,4151.79,4343.59,4584.62,4089.74,4615.90,4212.31,4226.67,4201.03,4269.74,4625.13,4378.46,0.0
7,4325.64,4006.67,4278.46,4143.08,4344.10,4583.08,4087.18,4614.87,4205.64,4230.26,4195.90,4266.67,4622.05,4380.51,0.0
8,4326.15,4010.77,4276.41,4139.49,4345.13,4584.10,4091.28,4608.21,4187.69,4229.74,4202.05,4273.85,4627.18,4389.74,0.0
9,4326.15,4011.28,4276.92,4142.05,4344.10,4582.56,4092.82,4608.72,4194.36,4228.72,4212.82,4277.95,4637.44,4393.33,0.0


### SVM Classification 

**The following attempts to find the best model using sklearn's SVM classifier. The different implementations attempted are outlined below: **<br><br>

*Note: all implementations use k = 10 fold cross validation*

**Part 1**: Data includes all 14 original features, using sklearn SVM (with default parameters)<br>
**Part 2**: Data feature dimensionality reduced with PCA, using sklearn's SVM (with default parameters) <br>
**Part 3**: Using PCA feature reduced data from Part 2, using sklearn's SVM (with tuned parameters) ¶

**First we extract our matrix of data including labels from the preprocessed pandas DataFrame above. After analyzing the data in the frame, there seems to be a large number of class '0' samples sectioned together, large number of class '1' samples sectioned together. To avoid biased folding later on, we shuffle the matrix holding all samples and their labels a few times. We then extract the data into a matrix and labels into a seperate vector.**

In [5]:
# Matrix including samples and labels from pandas DataFrame
EEG_Matrix = df.values

# Shuffle the matrix by rows a few times
for i in range(0, 100):
    np.random.shuffle(EEG_Matrix)

EEG_Data_Matrix = EEG_Matrix[:,:-1]
EEG_Data_Matrix = EEG_Data_Matrix.astype('float')
EEG_Data_Labels = EEG_Matrix[:,-1]
EEG_Data_Labels = EEG_Data_Labels.astype('int')

print("Dim(EEG_Data_Matrix) = ", EEG_Data_Matrix.shape)
print("Dim(EEG_Data_Labels) = ", EEG_Data_Labels.shape)

Dim(EEG_Data_Matrix) =  (14980, 14)
Dim(EEG_Data_Labels) =  (14980,)


**Next we define the main SVM training and testing method. The method below takes 13 arguments where arguments 1-9 are the fold indices to include for the training set, argument 10 is the fold index for the testing set. Argument 11 is a matrix holding all data samples without labels. Argument 12 is a vector of labels. This function needs this to partition into training and testing sets based on indices. Argument 13 is the sklearn classifier. **<br><br>
**This method trains and returns the testing accuracy** 

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Training and test SVM, return test score
def EEG_TrainTestSVM(f1, f2, f3, f4, f5, f6, f7, f8, f9, ftest, Data, Labels, clf):
    # Train Data and Labels
    train_data = np.c_[Data[f1 - 1].T, Data[f2 - 1].T, Data[f3 - 1].T, Data[f4 - 1].T, Data[f5 - 1].T, Data[f6 - 1].T, Data[f7 - 1].T, Data[f8 - 1].T, Data[f9 - 1].T].T
    train_labels = np.concatenate((Labels[f1 - 1], Labels[f2 - 1], Labels[f3 - 1], Labels[f4 - 1], Labels[f5 - 1], Labels[f6 - 1], Labels[f7 - 1], Labels[f8 - 1], Labels[f9 - 1]))
    
    # Test Data and Labels
    test_data = Data[ftest - 1]
    test_labels = Labels[ftest - 1]
    
    clf.fit(train_data, train_labels)
    
    # Test SVM
    predictions = [] # Stores classifier predictions
    for i in range(0, 1498):
        test_sample = test_data[i].reshape(1, -1)
        prediction = clf.predict(test_sample)
        predictions.append(prediction)
    
    accuracy = accuracy_score(test_labels, predictions)
    
    return accuracy

### Part 1: Original 14 features and default sklearn SVM

**Now we partition the data and labels into 10 folds for cross validation, 1498 samples per fold**

In [7]:
# Size per fold
f = 1498

# Partitioning data into 10 folds
EEG_Xfolds = np.array([EEG_Data_Matrix[:f], EEG_Data_Matrix[f:2*f], EEG_Data_Matrix[2*f:3*f], EEG_Data_Matrix[3*f:4*f], EEG_Data_Matrix[4*f:5*f], EEG_Data_Matrix[5*f:6*f], EEG_Data_Matrix[6*f:7*f], EEG_Data_Matrix[7*f:8*f], EEG_Data_Matrix[8*f:9*f], EEG_Data_Matrix[9*f:]])

# Partitioning labels into 10 folds
EEGlabel_fold1 = EEG_Data_Labels[0:f]
EEGlabel_fold2 = EEG_Data_Labels[f:2*f]
EEGlabel_fold3 = EEG_Data_Labels[2*f:3*f]
EEGlabel_fold4 = EEG_Data_Labels[3*f:4*f]
EEGlabel_fold5 = EEG_Data_Labels[4*f:5*f]
EEGlabel_fold6 = EEG_Data_Labels[5*f:6*f]
EEGlabel_fold7 = EEG_Data_Labels[6*f:7*f]
EEGlabel_fold8 = EEG_Data_Labels[7*f:8*f]
EEGlabel_fold9 = EEG_Data_Labels[8*f:9*f]
EEGlabel_fold10 = EEG_Data_Labels[9*f:]

EEG_Labels_folds = np.array([EEGlabel_fold1, EEGlabel_fold2, EEGlabel_fold3, EEGlabel_fold4, EEGlabel_fold5, EEGlabel_fold6, EEGlabel_fold7, EEGlabel_fold8, EEGlabel_fold9, EEGlabel_fold10])

**Now we define our sklearn SVM classifier with default parameters as an argument to our train/test method**

In [8]:
# Passed to our method in cross validation
clf = SVC(gamma = 'auto')

**Cross validation iterations**

In [9]:
# Store errors
EEG_SVM_accuracies = []

# Iteration 1
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 9 as training, fold 10 as test
EEGiteration1_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration1_accuracy)

# Iteration 2
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 10 as training, fold 9 as test
EEGiteration2_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 10, 9, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration2_accuracy)

# Iteration 3
# Using folds 1, 2, 3, 4, 5, 6, 7, 9, 10 as training, fold 8 as test
EEGiteration3_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 9, 10, 8, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration3_accuracy)

# Iteration 4
# Using folds 1, 2, 3, 4, 5, 6, 8, 9, 10 as training, fold 7 as test
EEGiteration4_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 8, 9, 10, 7, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration4_accuracy)

# Iteration 5
# Using folds 1, 2, 3, 4, 5, 7, 8, 9, 10 as training, fold 6 as test
EEGiteration5_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 7, 8, 9, 10, 6, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration5_accuracy)

# Iteration 6
# Using folds 1, 2, 3, 4, 6, 7, 8, 9, 10 as training, fold 5 as test
EEGiteration6_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 6, 7, 8, 9, 10, 5, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration6_accuracy)

# Iteration 7
# Using folds 1, 2, 3, 5, 6, 7, 8, 9, 10 as training, fold 4 as test
EEGiteration7_accuracy = EEG_TrainTestSVM(1, 2, 3, 5, 6, 7, 8, 9, 10, 4, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration7_accuracy)

# Iteration 8
# Using folds 1, 2, 4, 5, 6, 7, 8, 9, 10 as training, fold 3 as test
EEGiteration8_accuracy = EEG_TrainTestSVM(1, 2, 4, 5, 6, 7, 8, 9, 10, 3, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration8_accuracy)

# Iteration 9
# Using folds 1, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 2 as test
EEGiteration9_accuracy = EEG_TrainTestSVM(1, 3, 4, 5, 6, 7, 8, 9, 10, 2, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration9_accuracy)

# Iteration 10
# Using folds 2, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 1 as test
EEGiteration10_accuracy = EEG_TrainTestSVM(2, 3, 4, 5, 6, 7, 8, 9, 10, 1, EEG_Xfolds, EEG_Labels_folds, clf)
EEG_SVM_accuracies.append(EEGiteration10_accuracy)


print("Test Accuracies for Iterations 1-10: ", EEG_SVM_accuracies)

Test Accuracies for Iterations 1-10:  [0.5700934579439252, 0.5540720961281709, 0.548064085447263, 0.5674232309746329, 0.5527369826435247, 0.5427236315086782, 0.5487316421895861, 0.5433911882510013, 0.5534045393858478, 0.5313751668891856]


**Printing mean SVM accuracy across all k = 10 cross validation iterations**

In [10]:
print("Mean accuracy = ", np.mean(EEG_SVM_accuracies))

Mean accuracy =  0.5512016021361815


**First SVM Model Discussion**<br>
As seen above, the average accuracy across k = 10 fold cross validation is 0.55. This is not a good value and can be due to a few things. First, the SVM parameters for our training model were set to auto default. Since our data might be considered inseperable, it is crucial to wisely choose the kernel trick, and penalty parameter (C) to best seperate our data. This will allow for seperation of data in higher dimension but also tuning the C parameter creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane. Manipulating this can help take care of outliers closer to the seperating hyperplane for better classification. The last paramter to tune would be gamma. The range for gamma is 0.1 to 1.0 where 1.0 would usually cause overfitting of the model. In this case, we experiment with gamma set to 0.1 to avoid overfitting.

### Part 2: Using PCA for Top 3 Principal Components and Default Sklearn SVM

**First we show how many top k principal components will capture at least 95% of the data for feature reduction**<br>
FIX THIS

**Now we extract the data and labels from the already shuffled matrix from Part 1 (for better comparison between models). The data is mean centered. Then we compute the singular value decomposition to obtain the top three principal components from the sigma diagonal matrix. We use the three principal components to obtain the PCA based feauture representation X_Tilde of our mean centered data. Our data dimension is now 14980 by 3.** 

In [11]:
# Split data and labels    
EEG_PCA_Data = EEG_Matrix[:,:-1]
EEG_PCA_Data = EEG_PCA_Data.astype('float')
EEG_PCA_Labels = EEG_Matrix[:,-1]
EEG_PCA_Labels = EEG_PCA_Labels.astype('int')

print(EEG_PCA_Data.shape)
print(EEG_PCA_Labels.shape)

# Arranging each sample is a column in data matrix
X = EEG_PCA_Data.T # Data matrix

print(X.shape)

# Computation empirical mean - vector and tiled matrix
mean_vec = X.mean(axis = 1)
mean_mat = np.tile(mean_vec.reshape(X.shape[0],1),[1,X.shape[1]])

# Centered data matrix
centered_X = X - mean_mat

print(centered_X.shape)

# SVD of centered data
U, s, Vh = np.linalg.svd(centered_X)

# Top two principal components of centered data matrix
U_2 = U[:,0:3]

# Compute the PCA-based features
X_tilde = U_2.T@centered_X

X_tilde = X_tilde.T

print(X_tilde.shape)

(14980, 14)
(14980,)
(14, 14980)
(14, 14980)
(14980, 3)


**Now we partition our new feature reduced data and labels into folds, 1498 samples per fold**

In [12]:
# Partitioning data into 10 folds
f = 1498

PCA_EEG_Xfolds = np.array([X_tilde[:f], X_tilde[f:2*f], X_tilde[2*f:3*f], X_tilde[3*f:4*f], X_tilde[4*f:5*f], X_tilde[5*f:6*f], X_tilde[6*f:7*f], X_tilde[7*f:8*f], X_tilde[8*f:9*f], X_tilde[9*f:]])

# Partitioning labels into 10 folds
EEGlabel_fold1 = EEG_PCA_Labels[0:f]
EEGlabel_fold2 = EEG_PCA_Labels[f:2*f]
EEGlabel_fold3 = EEG_PCA_Labels[2*f:3*f]
EEGlabel_fold4 = EEG_PCA_Labels[3*f:4*f]
EEGlabel_fold5 = EEG_PCA_Labels[4*f:5*f]
EEGlabel_fold6 = EEG_PCA_Labels[5*f:6*f]
EEGlabel_fold7 = EEG_PCA_Labels[6*f:7*f]
EEGlabel_fold8 = EEG_PCA_Labels[7*f:8*f]
EEGlabel_fold9 = EEG_PCA_Labels[8*f:9*f]
EEGlabel_fold10 = EEG_PCA_Labels[9*f:]

EEG_PCALabels_folds = np.array([EEGlabel_fold1, EEGlabel_fold2, EEGlabel_fold3, EEGlabel_fold4, EEGlabel_fold5, EEGlabel_fold6, EEGlabel_fold7, EEGlabel_fold8, EEGlabel_fold9, EEGlabel_fold10])

**Now we again define our sklearn SVM classifier with default parameters as an argument to our train/test method**

In [13]:
# Passed to our method in cross validation
clf = SVC(gamma = 'auto')

**Cross validation iterations using default SVM w/ PCA Dimension Reduced Data**

In [14]:
# Store errors
EEG_SVM2_accuracies = []

# Iteration 1
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 9 as training, fold 10 as test
EEGiteration1_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration1_accuracy)

# Iteration 2
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 10 as training, fold 9 as test
EEGiteration2_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 10, 9, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration2_accuracy)

# Iteration 3
# Using folds 1, 2, 3, 4, 5, 6, 7, 9, 10 as training, fold 8 as test
EEGiteration3_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 9, 10, 8, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration3_accuracy)

# Iteration 4
# Using folds 1, 2, 3, 4, 5, 6, 8, 9, 10 as training, fold 7 as test
EEGiteration4_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 8, 9, 10, 7, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration4_accuracy)

# Iteration 5
# Using folds 1, 2, 3, 4, 5, 7, 8, 9, 10 as training, fold 6 as test
EEGiteration5_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 7, 8, 9, 10, 6, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration5_accuracy)

# Iteration 6
# Using folds 1, 2, 3, 4, 6, 7, 8, 9, 10 as training, fold 5 as test
EEGiteration6_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 6, 7, 8, 9, 10, 5, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration6_accuracy)

# Iteration 7
# Using folds 1, 2, 3, 5, 6, 7, 8, 9, 10 as training, fold 4 as test
EEGiteration7_accuracy = EEG_TrainTestSVM(1, 2, 3, 5, 6, 7, 8, 9, 10, 4, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration7_accuracy)

# Iteration 8
# Using folds 1, 2, 4, 5, 6, 7, 8, 9, 10 as training, fold 3 as test
EEGiteration8_accuracy = EEG_TrainTestSVM(1, 2, 4, 5, 6, 7, 8, 9, 10, 3, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration8_accuracy)

# Iteration 9
# Using folds 1, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 2 as test
EEGiteration9_accuracy = EEG_TrainTestSVM(1, 3, 4, 5, 6, 7, 8, 9, 10, 2, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration9_accuracy)

# Iteration 10
# Using folds 2, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 1 as test
EEGiteration10_accuracy = EEG_TrainTestSVM(2, 3, 4, 5, 6, 7, 8, 9, 10, 1, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf)
EEG_SVM2_accuracies.append(EEGiteration10_accuracy)

print("Second Model: Test Accuracies for Iterations 1-10: ", EEG_SVM2_accuracies)

Second Model: Test Accuracies for Iterations 1-10:  [0.6068090787716955, 0.5921228304405874, 0.5934579439252337, 0.6101468624833111, 0.5967957276368492, 0.5921228304405874, 0.5961281708945261, 0.5907877169559412, 0.6088117489986649, 0.5801068090787717]


**Printing mean SVM accuracy across all k = 10 cross validation iterations**

In [15]:
print("Mean accuracy = ", np.mean(EEG_SVM2_accuracies))

Mean accuracy =  0.5967289719626169


**Second SVM Model Discussion**<br>
FIX THIS<br>
As seen above, the average accuracy across k = 10 fold cross validation is 0.55. This is not a good value and can be due to a few things. First, the SVM parameters for our training model were set to auto default. Since our data might be considered inseperable, it is crucial to wisely choose the kernel trick, and penalty parameter (C) to best seperate our data. This will allow for seperation of data in higher dimension but also tuning the C parameter creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane. Manipulating this can help take care of outliers closer to the seperating hyperplane for better classification. The last paramter to tune would be gamma. The range for gamma is 0.1 to 1.0 where 1.0 would usually cause overfitting of the model. In this case, we experiment with gamma set to 0.1 to avoid overfitting.

### Part 3: Using PCA Feature Reduced Data from Part 2 and Tuned Parameters Sklearn SVM

**For this model, we do not need to manipulate the data matrix or vector of labels since we will just reuse from Part 2. Essentially, the only difference here will be different sklearn SVM parameters instead of the default. See the tuned classifier below:**

In [16]:
# Classifier w/ tuned parameters
clf3 = SVC(C=1.0, gamma = 0.1, kernel = 'rbf')

**Cross validation iterations using SVM w/ tuned parameters**

In [17]:
# Store errors
EEG_SVM3_accuracies = []

# Iteration 1
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 9 as training, fold 10 as test
EEGiteration1_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration1_accuracy)

# Iteration 2
# Using folds 1, 2, 3, 4, 5, 6, 7, 8, 10 as training, fold 9 as test
EEGiteration2_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 8, 10, 9, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration2_accuracy)

# Iteration 3
# Using folds 1, 2, 3, 4, 5, 6, 7, 9, 10 as training, fold 8 as test
EEGiteration3_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 7, 9, 10, 8, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration3_accuracy)

# Iteration 4
# Using folds 1, 2, 3, 4, 5, 6, 8, 9, 10 as training, fold 7 as test
EEGiteration4_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 6, 8, 9, 10, 7, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration4_accuracy)

# Iteration 5
# Using folds 1, 2, 3, 4, 5, 7, 8, 9, 10 as training, fold 6 as test
EEGiteration5_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 5, 7, 8, 9, 10, 6, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration5_accuracy)

# Iteration 6
# Using folds 1, 2, 3, 4, 6, 7, 8, 9, 10 as training, fold 5 as test
EEGiteration6_accuracy = EEG_TrainTestSVM(1, 2, 3, 4, 6, 7, 8, 9, 10, 5, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration6_accuracy)

# Iteration 7
# Using folds 1, 2, 3, 5, 6, 7, 8, 9, 10 as training, fold 4 as test
EEGiteration7_accuracy = EEG_TrainTestSVM(1, 2, 3, 5, 6, 7, 8, 9, 10, 4, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration7_accuracy)

# Iteration 8
# Using folds 1, 2, 4, 5, 6, 7, 8, 9, 10 as training, fold 3 as test
EEGiteration8_accuracy = EEG_TrainTestSVM(1, 2, 4, 5, 6, 7, 8, 9, 10, 3, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration8_accuracy)

# Iteration 9
# Using folds 1, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 2 as test
EEGiteration9_accuracy = EEG_TrainTestSVM(1, 3, 4, 5, 6, 7, 8, 9, 10, 2, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration9_accuracy)

# Iteration 10
# Using folds 2, 3, 4, 5, 6, 7, 8, 9, 10 as training, fold 1 as test
EEGiteration10_accuracy = EEG_TrainTestSVM(2, 3, 4, 5, 6, 7, 8, 9, 10, 1, PCA_EEG_Xfolds, EEG_PCALabels_folds, clf3)
EEG_SVM3_accuracies.append(EEGiteration10_accuracy)

print("Third Model: Test Accuracies for Iterations 1-10: ", EEG_SVM3_accuracies)

Third Model: Test Accuracies for Iterations 1-10:  [0.6535380507343124, 0.6602136181575434, 0.6475300400534045, 0.6455273698264352, 0.6522029372496663, 0.6448598130841121, 0.6515353805073432, 0.6615487316421896, 0.6468624833110814, 0.6468624833110814]


**Printing mean SVM accuracy across all k = 10 cross validation iterations**

In [18]:
print("Mean accuracy = ", np.mean(EEG_SVM3_accuracies))

Mean accuracy =  0.651068090787717


**Third SVM Model Discussion**<br>
FIX THIS<br>
As seen above, the average accuracy across k = 10 fold cross validation is 0.55. This is not a good value and can be due to a few things. First, the SVM parameters for our training model were set to auto default. Since our data might be considered inseperable, it is crucial to wisely choose the kernel trick, and penalty parameter (C) to best seperate our data. This will allow for seperation of data in higher dimension but also tuning the C parameter creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane. Manipulating this can help take care of outliers closer to the seperating hyperplane for better classification. The last paramter to tune would be gamma. The range for gamma is 0.1 to 1.0 where 1.0 would usually cause overfitting of the model. In this case, we experiment with gamma set to 0.1 to avoid overfitting.

### SVM Analysis

In [ ]:
# Talk about the SVM parameters and their affect on the classification test accuracy
# Also mention also possible factors to this low score